In [3]:
import pandas as pd
import glob
import os
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import get_constellation

import numpy as np

import matplotlib.pyplot as plt

# Pré-traitement des exoplanètes

## Ajout des coodonnées x0,y0,z0

In [4]:
df = pd.read_csv('.\\Tables_brutes\\planetes_table_limite.csv', header=41,sep=',')

In [5]:
df.head()

,loc_rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,disc_instrument,pl_orbper,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_eqt,st_spectype,st_teff,st_rad,st_mass,st_met,st_metratio,st_lum,st_logg,st_age,st_dens,ra,dec,sy_dist
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,323.21000,12.2,1.09,4914.89849,15.464,14.90,0.238,NaN,G8 III,4874.0,13.76,2.09,-0.260,[Fe/H],1.978,2.45,NaN,NaN,185.178779,17.793252,93.1846
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,516.21997,12.3,1.09,4684.81420,14.740,13.80,0.080,NaN,K4 III,4213.0,29.79,2.78,-0.020,[Fe/H],2.430,1.93,1.56,NaN,229.274595,71.823943,125.3210
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,186.76000,13.1,1.16,1131.15130,3.559,2.76,0.000,NaN,K0 III,4888.0,11.55,1.78,-0.210,[Fe/H],1.840,2.55,4.50,NaN,352.824150,39.235837,75.4392
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,Radial Velocity,2002,W. M. Keck Observatory,10 m Keck I Telescope,HIRES Spectrometer,1765.03890,12.6,1.12,2559.47216,8.053,7.03,0.373,NaN,K0 V,5338.0,0.93,0.91,0.405,[Fe/H],-0.153,4.45,3.90,1.27393,242.602101,43.816362,17.9323
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,Radial Velocity,1996,Multiple Observatories,Multiple Telescopes,Multiple Instruments,798.50000,13.5,1.20,565.73740,1.780,1.26,0.680,NaN,G3 V,5750.0,1.13,1.08,0.060,[Fe/H],0.097,4.36,7.40,1.01103,295.465642,50.516824,21.1397


In [6]:
df.shape

(5539, 36)

In [7]:
df.columns

Index(['loc_rowid', 'pl_name', 'hostname', 'pl_letter', 'hd_name', 'hip_name',
       'tic_id', 'gaia_id', 'sy_snum', 'sy_pnum', 'discoverymethod',
       'disc_year', 'disc_facility', 'disc_telescope', 'disc_instrument',
       'pl_orbper', 'pl_rade', 'pl_radj', 'pl_bmasse', 'pl_bmassj', 'pl_dens',
       'pl_orbeccen', 'pl_eqt', 'st_spectype', 'st_teff', 'st_rad', 'st_mass',
       'st_met', 'st_metratio', 'st_lum', 'st_logg', 'st_age', 'st_dens', 'ra',
       'dec', 'sy_dist'],
      dtype='object')

In [8]:
df[['pl_name', 'dec', 'ra', 'sy_dist']]

,pl_name,dec,ra,sy_dist
0,11 Com b,17.793252,185.178779,93.1846
1,11 UMi b,71.823943,229.274595,125.3210
2,14 And b,39.235837,352.824150,75.4392
3,14 Her b,43.816362,242.602101,17.9323
4,16 Cyg B b,50.516824,295.465642,21.1397
...,...,...,...,...
5534,ups And b,41.403815,24.198353,13.4054
5535,ups And c,41.403815,24.198353,13.4054
5536,ups And d,41.403815,24.198353,13.4054
5537,ups Leo b,-0.823564,174.237219,52.5973


In [9]:
declinaison = np.array((df[['dec']])*np.pi/180.)
right_asc = np.array((df[['ra']])*np.pi/180.)
dist_sol = np.array(df[['sy_dist']])

In [10]:
x0 = dist_sol * np.cos(declinaison) * np.cos(right_asc)
y0 = dist_sol * np.cos(declinaison) * np.sin(right_asc)
z0 = dist_sol * np.sin(declinaison)

coordonnees = pd.DataFrame({'x0':x0[:,0], 'y0':y0[:,0], 'z0':z0[:,0]})

In [11]:
print(coordonnees.isna().sum())
print(coordonnees.shape)

x0    21
y0    21
z0    21
dtype: int64
(5539, 3)


In [12]:
dfnew = pd.concat([df, coordonnees], axis=1)
dfnew.head()

,loc_rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,disc_instrument,pl_orbper,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_eqt,st_spectype,st_teff,st_rad,st_mass,st_met,st_metratio,st_lum,st_logg,st_age,st_dens,ra,dec,sy_dist,x0,y0,z0
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,323.21000,12.2,1.09,4914.89849,15.464,14.90,0.238,NaN,G8 III,4874.0,13.76,2.09,-0.260,[Fe/H],1.978,2.45,NaN,NaN,185.178779,17.793252,93.1846,-88.364958,-8.008843,28.475644
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,516.21997,12.3,1.09,4684.81420,14.740,13.80,0.080,NaN,K4 III,4213.0,29.79,2.78,-0.020,[Fe/H],2.430,1.93,1.56,NaN,229.274595,71.823943,125.3210,-25.505212,-29.625963,119.067794
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,186.76000,13.1,1.16,1131.15130,3.559,2.76,0.000,NaN,K0 III,4888.0,11.55,1.78,-0.210,[Fe/H],1.840,2.55,4.50,NaN,352.824150,39.235837,75.4392,57.973692,-7.298956,47.716341
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,Radial Velocity,2002,W. M. Keck Observatory,10 m Keck I Telescope,HIRES Spectrometer,1765.03890,12.6,1.12,2559.47216,8.053,7.03,0.373,NaN,K0 V,5338.0,0.93,0.91,0.405,[Fe/H],-0.153,4.45,3.90,1.27393,242.602101,43.816362,17.9323,-5.954231,-11.487907,12.415415
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,Radial Velocity,1996,Multiple Observatories,Multiple Telescopes,Multiple Instruments,798.50000,13.5,1.20,565.73740,1.780,1.26,0.680,NaN,G3 V,5750.0,1.13,1.08,0.060,[Fe/H],0.097,4.36,7.40,1.01103,295.465642,50.516824,21.1397,5.779530,-12.135760,16.315860


## Separation des informations du type spectral

In [11]:
# dfnew[['st_specclass', 'st_specsubdiv', 'st_lumclass']] = dfnew.st_spectype.str.extract(r'([OBAFGKM])(\d|\d\.\d)(?:\s)(III|II|IV|I|V)')

dfnew[['st_specclass', 'st_specsubdiv']] = dfnew.st_spectype.str.extract(r'([OBAFGKM])(.{0,})')
dfnew[['st_specsubdiv', 'st_lumclass']] = dfnew.st_specsubdiv.str.extract(r'(\d\.\d|\d)(.{0,})')
dfnew[['st_lumclass']] = dfnew.st_lumclass.str.extract(r'(III|II|IV|I|V)')
dfnew.loc[:100]

,loc_rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,disc_instrument,pl_orbper,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_eqt,st_spectype,st_teff,st_rad,st_mass,st_met,st_metratio,st_lum,st_logg,st_age,st_dens,ra,dec,sy_dist,x0,y0,z0,st_specclass,st_specsubdiv,st_lumclass
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,323.21000,12.200,1.090,4914.89849,15.464,14.900,0.238,NaN,G8 III,4874.0,13.76,2.09,-0.260,[Fe/H],1.978,2.45,NaN,NaN,185.178779,17.793252,93.184600,-88.364958,-8.008843,28.475644,G,8,III
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,516.21997,12.300,1.090,4684.81420,14.740,13.800,0.080,NaN,K4 III,4213.0,29.79,2.78,-0.020,[Fe/H],2.430,1.93,1.56,NaN,229.274595,71.823943,125.321000,-25.505212,-29.625963,119.067794,K,4,III
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,186.76000,13.100,1.160,1131.15130,3.559,2.760,0.000,NaN,K0 III,4888.0,11.55,1.78,-0.210,[Fe/H],1.840,2.55,4.50,NaN,352.824150,39.235837,75.439200,57.973692,-7.298956,47.716341,K,0,III
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,Radial Velocity,2002,W. M. Keck Observatory,10 m Keck I Telescope,HIRES Spectrometer,1765.03890,12.600,1.120,2559.47216,8.053,7.030,0.373,NaN,K0 V,5338.0,0.93,0.91,0.405,[Fe/H],-0.153,4.45,3.90,1.27393,242.602101,43.816362,17.932300,-5.954231,-11.487907,12.415415,K,0,V
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,Radial Velocity,1996,Multiple Observatories,Multiple Telescopes,Multiple Instruments,798.50000,13.500,1.200,565.73740,1.780,1.260,0.680,NaN,G3 V,5750.0,1.13,1.08,0.060,[Fe/H],0.097,4.36,7.40,1.01103,295.465642,50.516824,21.139700,5.779530,-12.135760,16.315860,G,3,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,CoRoT-21 b,CoRoT-21,b,NaN,NaN,TIC 280420719,Gaia DR2 3107471240236703104,1,1,Transit,2012,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,2.72474,14.572,1.300,718.29580,2.260,1.360,0.000,NaN,F8 IV,6200.0,1.95,1.29,0.000,[Fe/H],0.705,3.70,4.10,0.25500,101.052621,-0.299154,1398.834197,-268.167472,1372.869249,-7.303601,F,8,IV
97,98,CoRoT-22 b,CoRoT-22,b,NaN,NaN,TIC 323094535,Gaia DR2 4285572454508522496,1,1,Transit,2014,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,9.75598,4.880,0.435,12.20000,0.038,0.249,0.077,885.0,G3 V,5939.0,1.14,1.10,0.170,[Fe/H],0.014,4.36,3.30,1.02700,280.667139,6.219167,627.158000,115.405636,-612.692978,67.941229,G,3,V
98,99,CoRoT-23 b,CoRoT-23,b,NaN,NaN,TIC 108155949,Gaia DR2 4283597937777249408,1,1,Transit,2011,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,3.63130,11.770,1.050,889.90000,2.800,3.000,0.160,1660.0,G0 V,5900.0,1.61,1.14,0.050,[Fe/H],0.087,4.30,7.20,0.50000,279.782659,4.357752,1060.970000,179.749579,-1042.520217,80.616540,G,0,V
99,100,CoRoT-24 b,CoRoT-24,b,NaN,NaN,TIC 36105482,Gaia DR2 3105404467618982272,1,2,Transit,2014,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,5.11340,3.700,0.330,5.70000,0.018,0.900,0.000,1070.0,K1 V,4950.0,0.86,0.91,0.300,[Fe/H],-0.601,4.55,11.00,1.94863,101.922543,-3.719289,591.555000,-121.951461,577.574810,-38.373145,K,1,V


In [30]:
# dfnew.to_csv('.\\Tables_traitees\\planetes_avec_repere_cartesien.csv', sep = ',', index = False)

## Détermination des constellations à partir des coordonnées (planètes, étoiles)

In [13]:
Planets = pd.read_csv(r'.\\Tables_traitees\\planetes_avec_repere_cartesien.csv', header=0,sep=',')

In [14]:
Planets.shape

(5539, 42)

In [15]:
Planets[:10]

,loc_rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,disc_instrument,pl_orbper,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_eqt,st_spectype,st_teff,st_rad,st_mass,st_met,st_metratio,st_lum,st_logg,st_age,st_dens,ra,dec,sy_dist,x0,y0,z0,st_specclass,st_specsubdiv,st_lumclass
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,323.21000,12.200,1.090,4914.89849,15.464,14.900,0.238,NaN,G8 III,4874.0,13.76,2.09,-0.260,[Fe/H],1.978,2.45,NaN,NaN,185.178779,17.793252,93.1846,-88.364958,-8.008843,28.475644,G,8.0,III
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,516.21997,12.300,1.090,4684.81420,14.740,13.800,0.080,NaN,K4 III,4213.0,29.79,2.78,-0.020,[Fe/H],2.430,1.93,1.560,NaN,229.274595,71.823943,125.3210,-25.505212,-29.625963,119.067794,K,4.0,III
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,186.76000,13.100,1.160,1131.15130,3.559,2.760,0.000,NaN,K0 III,4888.0,11.55,1.78,-0.210,[Fe/H],1.840,2.55,4.500,NaN,352.824150,39.235837,75.4392,57.973692,-7.298956,47.716341,K,0.0,III
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,Radial Velocity,2002,W. M. Keck Observatory,10 m Keck I Telescope,HIRES Spectrometer,1765.03890,12.600,1.120,2559.47216,8.053,7.030,0.373,NaN,K0 V,5338.0,0.93,0.91,0.405,[Fe/H],-0.153,4.45,3.900,1.27393,242.602101,43.816362,17.9323,-5.954231,-11.487907,12.415415,K,0.0,V
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,Radial Velocity,1996,Multiple Observatories,Multiple Telescopes,Multiple Instruments,798.50000,13.500,1.200,565.73740,1.780,1.260,0.680,NaN,G3 V,5750.0,1.13,1.08,0.060,[Fe/H],0.097,4.36,7.400,1.01103,295.465642,50.516824,21.1397,5.779530,-12.135760,16.315860,G,3.0,V
5,6,17 Sco b,17 Sco,b,NaN,HIP 79540,TIC 135596590,Gaia DR2 4342464209753404416,1,1,Radial Velocity,2020,Lick Observatory,Coude Auxiliary Telescope,Hamilton Echelle Spectrograph,578.38000,12.900,1.150,1373.01872,4.320,3.510,0.060,NaN,K3 III,4157.0,25.92,1.22,-0.010,[Fe/H],2.254,1.70,5.130,NaN,243.462087,-11.837791,124.9530,-54.640410,-109.410346,-25.633063,K,3.0,III
6,7,18 Del b,18 Del,b,HD 199665,HIP 103527,TIC 354489950,Gaia DR2 1756741374681702784,2,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,982.85000,12.500,1.120,2926.24614,9.207,8.230,0.024,NaN,G6 III,4980.0,8.80,2.10,-0.060,[Fe/H],1.634,2.87,1.100,NaN,314.607838,10.839138,76.2220,52.571961,-53.296592,14.333719,G,6.0,III
7,8,1RXS J160929.1-210524 b,1RXS J160929.1-210524,b,NaN,NaN,TIC 48964101,Gaia DR2 6243841249531772800,1,1,Imaging,2008,Gemini Observatory,8.19 m Gemini North Telescope,NIRI Camera,NaN,18.647,1.664,3000.00000,8.000,2.540,NaN,1700.0,K7 V,4060.0,1.31,0.85,NaN,NaN,-0.370,4.00,0.005,0.38789,242.376234,-21.083140,139.1350,-60.193374,-115.023075,-50.049957,K,7.0,V
8,9,24 Boo b,24 Boo,b,HD 127243,HIP 70791,TIC 441712711,Gaia DR2 1603662211901473408,1,1,Radial Velocity,2018,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,30.33000,13.900,1.240,280.64248,0.883,0.574,0.032,NaN,G3 IV,4816.0,12.19,1.05,-0.780,[Fe/H],1.877,2.28,6.920,NaN,217.155524,49.844649,95.9863,-49.332600,-37.385160,73.362212,G,3.0,IV
9,10,24 Sex b,24 Sex,b,HD 90043,HIP 50887,TIC 1713457,Gaia DR2 3830897080395058048,1,2,Radial Velocity,2010,Lick Observatory,3.0 m C. Donald Shane Telescope,Hamilton Echelle Spectrograph,452.80000,13.400,1.190,632.46000,1.990,1.440,0.090,NaN,K0 IV,5098.0,4.90,1.54,-0.030,[Fe/H],1.164,3.50,2.700,NaN,155.

In [16]:
sel_col = ['pl_name', 'pl_bmasse','pl_rade',
           'pl_orbper',          
           'hostname', 'hd_name', 'hip_name',
           'gaia_id', 
           'sy_snum', 'st_mass',  'st_rad', 'st_age',
           'discoverymethod', 'disc_year', 'disc_facility',
           'st_spectype',
           'ra','dec',
           'con'
          ]

In [17]:
def get_constellation_name(ra, dec):
    # Create a SkyCoord object with the given RA and Dec
    coordinates = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, frame='icrs')

    # Get the constellation name
    constellation_name = get_constellation(coordinates)

    return constellation_name

# Example usage
row = 3
ra = Planets.loc[row,'ra']# 10.68479  # Replace with the actual RA of your object
dec = Planets.loc[row,'dec']#41.26906  # Replace with the actual Dec of your object

constellation = get_constellation_name(ra, dec)
print(f"The object is in the constellation:{ra=} {dec=} {constellation}")

The object is in the constellation:ra=242.6021014 dec=43.8163621 Hercules


In [36]:
get_constellation_name(270, 60)

'Draco'

In [37]:
Planets['con']=Planets.apply(lambda pl: get_constellation_name(pl['ra'],pl['dec']), axis=1)

In [38]:
Planets.loc[:100]

,loc_rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,disc_instrument,pl_orbper,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_eqt,st_spectype,st_teff,st_rad,st_mass,st_met,st_metratio,st_lum,st_logg,st_age,st_dens,ra,dec,sy_dist,x0,y0,z0,st_specclass,st_specsubdiv,st_lumclass,con
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,323.21000,12.200,1.090,4914.89849,15.464,14.900,0.238,NaN,G8 III,4874.0,13.76,2.09,-0.260,[Fe/H],1.978,2.45,NaN,NaN,185.178779,17.793252,93.184600,-88.364958,-8.008843,28.475644,G,8.0,III,Coma Berenices
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,516.21997,12.300,1.090,4684.81420,14.740,13.800,0.080,NaN,K4 III,4213.0,29.79,2.78,-0.020,[Fe/H],2.430,1.93,1.56,NaN,229.274595,71.823943,125.321000,-25.505212,-29.625963,119.067794,K,4.0,III,Ursa Minor
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,186.76000,13.100,1.160,1131.15130,3.559,2.760,0.000,NaN,K0 III,4888.0,11.55,1.78,-0.210,[Fe/H],1.840,2.55,4.50,NaN,352.824150,39.235837,75.439200,57.973692,-7.298956,47.716341,K,0.0,III,Andromeda
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,Radial Velocity,2002,W. M. Keck Observatory,10 m Keck I Telescope,HIRES Spectrometer,1765.03890,12.600,1.120,2559.47216,8.053,7.030,0.373,NaN,K0 V,5338.0,0.93,0.91,0.405,[Fe/H],-0.153,4.45,3.90,1.27393,242.602101,43.816362,17.932300,-5.954231,-11.487907,12.415415,K,0.0,V,Hercules
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,Radial Velocity,1996,Multiple Observatories,Multiple Telescopes,Multiple Instruments,798.50000,13.500,1.200,565.73740,1.780,1.260,0.680,NaN,G3 V,5750.0,1.13,1.08,0.060,[Fe/H],0.097,4.36,7.40,1.01103,295.465642,50.516824,21.139700,5.779530,-12.135760,16.315860,G,3.0,V,Cygnus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,CoRoT-21 b,CoRoT-21,b,NaN,NaN,TIC 280420719,Gaia DR2 3107471240236703104,1,1,Transit,2012,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,2.72474,14.572,1.300,718.29580,2.260,1.360,0.000,NaN,F8 IV,6200.0,1.95,1.29,0.000,[Fe/H],0.705,3.70,4.10,0.25500,101.052621,-0.299154,1398.834197,-268.167472,1372.869249,-7.303601,F,8.0,IV,Monoceros
97,98,CoRoT-22 b,CoRoT-22,b,NaN,NaN,TIC 323094535,Gaia DR2 4285572454508522496,1,1,Transit,2014,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,9.75598,4.880,0.435,12.20000,0.038,0.249,0.077,885.0,G3 V,5939.0,1.14,1.10,0.170,[Fe/H],0.014,4.36,3.30,1.02700,280.667139,6.219167,627.158000,115.405636,-612.692978,67.941229,G,3.0,V,Serpens
98,99,CoRoT-23 b,CoRoT-23,b,NaN,NaN,TIC 108155949,Gaia DR2 4283597937777249408,1,1,Transit,2011,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,3.63130,11.770,1.050,889.90000,2.800,3.000,0.160,1660.0,G0 V,5900.0,1.61,1.14,0.050,[Fe/H],0.087,4.30,7.20,0.50000,279.782659,4.357752,1060.970000,179.749579,-1042.520217,80.616540,G,0.0,V,Serpens
99,100,CoRoT-24 b,CoRoT-24,b,NaN,NaN,TIC 36105482,Gaia DR2 3105404467618982272,1,2,Transit,2014,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,5.11340,3.700,0.330,5.70000,0.018,0.900,0.000,1070.0,K1 V,4950.0,0.86,0.91,0.300,[Fe/H],-0.601,4.55,11.00,1.94863,101.922543,-3.719289,591.555000,-121.951461,577.574810,-38.373145,K,1.0,V,Monoceros


In [39]:
Planets.loc[Planets['con']=='NaN',sel_col].head(100)

,pl_name,pl_bmasse,pl_rade,pl_orbper,hostname,hd_name,hip_name,gaia_id,sy_snum,st_mass,st_rad,st_age,discoverymethod,disc_year,disc_facility,st_spectype,ra,dec,con


In [40]:
Planets.con.value_counts()

Cygnus                 1652
Lyra                   1046
Draco                   243
Virgo                   190
Sagittarius             175
Cancer                  129
Aquarius                122
Scorpius                 97
Taurus                   87
Cetus                    85
Pisces                   79
Leo                      77
Eridanus                 72
Ophiucus                 67
Hydra                    64
Centaurus                64
Ursa Major               58
Hercules                 48
Libra                    42
Andromeda                36
Cassiopeia               35
Sextans                  33
Boötes                   32
Puppis                   32
Monoceros                31
Lepus                    30
Pegasus                  30
Orion                    29
Carina                   28
Phoenix                  28
Cepheus                  27
Hydrus                   26
Vela                     26
Serpens                  25
Canis Major              25
Fornax              

In [41]:
# Planets.to_csv(r'.\\Tables_traitees\\planetes_avec_repere_cartesien_et_constellations.csv', index=False,sep=',')

## Voisinages des constellations

### Définition d'une liste de couple de voisins

In [ ]:
dec_table = np.arange(-90, 91, 3)
ra_table = np.arange(-180,181,3)

In [ ]:
dec_table

In [ ]:
len(dec_table)

In [ ]:
ra_table

In [ ]:
len(ra_table)

In [ ]:
# Construction de la carte 
Const_map = np.full((len(dec_table),len(ra_table)),'12345678912345678912')
for j in range(len(ra_table)):
    for i in range(len(dec_table)):
        Const_map[i,j] = get_constellation_name(ra_table[j], dec_table[i])

In [ ]:
Const_map

In [ ]:
Const_map.shape

In [ ]:
Const_neighbours = list()
for i in range(Const_map.shape[0]):
    for j in range(Const_map.shape[1]-1):
        const1 = Const_map[i,j]
        const2 = Const_map[i,j+1]
        if const1 != const2: 
            Const_neighbours.append((const1,const2))
for j in range(Const_map.shape[1]-1):
    for i in range(Const_map.shape[0]-1):
        const1 = Const_map[i,j]
        const2 = Const_map[i+1,j]
        if const1 != const2: 
            Const_neighbours.append((const1,const2))

In [ ]:
len(Const_neighbours)

In [ ]:
Const_neighbours_set = set(Const_neighbours)
len(Const_neighbours_set)

In [ ]:
Const_neighbours_set

### Extraction des relations uniques de voisinage entre constellations

In [ ]:
Const_neighbours_set_un_lien = set()
for pair in Const_neighbours_set:
    if pair[0]<pair[1]:
        Const_neighbours_set_un_lien.add((pair[0],pair[1]))
    else:
        Const_neighbours_set_un_lien.add((pair[1],pair[0]))

In [ ]:
len(Const_neighbours_set_un_lien)

In [ ]:
Const_neighbours_set_un_lien

In [ ]:
df_relation_un_lien = pd.DataFrame(Const_neighbours_set_un_lien, columns=['con1','con2'])
df_relation_un_lien

In [ ]:
df_relation_un_lien[(df_relation_un_lien.con1=='Pegasus') | (df_relation_un_lien.con2=='Pegasus')]

In [ ]:
Constellations_list = sorted(list(set(df_relation_un_lien.con1.to_list()+df_relation_un_lien.con2.to_list())))
Constellations_list

In [ ]:
len(Constellations_list)

In [ ]:
# pd.DataFrame(Const_neighbours_set_un_lien, columns=['con1','con2']).to_csv(r'.\\Tables_traitees\\Constellations_voisines_une_seule_relation.csv', index=False,sep=',')

# Preparation de la Visibilité et alignement des noms des Constellations

In [ ]:
df_cons = pd.read_csv(r'.\\Tables_brutes\\List_of_constellations.csv', header=0,sep=',')

In [ ]:
df_cons.shape

In [ ]:
df_cons.columns

In [ ]:
df_cons

In [ ]:
def visibility(sector):
    # print(sector)
    Lat_N = int(sector.replace('–','-').split('-')[0].replace('°','').replace('N',''))
    Lat_S = -int(sector.replace('–','-').split('-')[1].replace('°','').replace('S',''))
    return Lat_N,Lat_S

In [ ]:
df_cons['Visibilite_Nord']=df_cons.apply(lambda x: visibility(x['Visible between latitudes'])[0],axis=1)
df_cons['Visibilite_Sud']=df_cons.apply(lambda x: visibility(x['Visible between latitudes'])[1],axis=1)

In [ ]:
df_cons.rename(columns ={'Area (square degrees)':'Area_square_degrees'}, inplace=True)

In [ ]:
df_cons

In [ ]:
df_cons['Constellation'].unique()

In [ ]:
# Noms différents de constellations dans les 2 bases
delta = [c for c in df_cons['Constellation'].unique() if c not in Constellations_list]
delta

In [ ]:
# Alignement sur les noms de la librairies Astropy
df_cons.loc[df_cons['Constellation']== 'Chamaeleon','Constellation'] = 'Chamaleon'  
df_cons.loc[df_cons['Constellation']== 'Crux','Constellation'] = 'Crux '
df_cons.loc[df_cons['Constellation']== 'Ophiuchus','Constellation'] = 'Ophiucus'
df_cons.loc[df_cons['Constellation']== 'Piscis Austrinus','Constellation'] = 'Pisces Austrinus'

In [ ]:
# df_cons.to_csv(r'.\\Tables_traitees\\Visibilité_Constellations.csv', index=False,sep=',')

# Liste des observatoires

In [ ]:
df_obs_latlon = pd.read_csv(r'.\\Tables_brutes\\list_observatories.csv', header=0,sep=';')

In [ ]:
df_obs_latlon.shape

In [ ]:
df_obs_latlon.head()

## Conversion des coordonnées en decimal

In [ ]:
def dms_to_decimal_degrees(val):
    try:
        val = val.replace("\'",'"')
        direction = val.split('"')[2].strip()
        seconds = val.split('"')[1]
        minutes = val.split('"')[0].split('°')[1]
        degrees = val.split('"')[0].split('°')[0]

        # Calculate the decimal degrees
        decimal_degrees = float(degrees) + float(minutes)/60 + float(seconds)/3600
        # Apply the direction (positive for 'N', negative for 'S')
        if direction == 'S' or direction == 'W':
            decimal_degrees = -decimal_degrees
        output = (decimal_degrees, direction)
    except:
        output = ('Nan','Nan')
    return output

In [ ]:
latitude_decimal = dms_to_decimal_degrees(df_obs_latlon.iloc[0,3].replace("\'",'"'))
print(f"Latitude in Decimal Degrees: {latitude_decimal}")

In [ ]:
df_obs_latlon['Lat_dec']= df_obs_latlon['Latitude (°)'].apply(lambda x: dms_to_decimal_degrees(x)[0])
df_obs_latlon['Hemisphere']= df_obs_latlon['Latitude (°)'].apply(lambda x: dms_to_decimal_degrees(x)[1])
df_obs_latlon['Lon_dec']= df_obs_latlon['Longitude (°)'].apply(lambda x: dms_to_decimal_degrees(x)[0])
df_obs_latlon['Lat_Lon']= df_obs_latlon.apply(lambda x: (x.Lat_dec,x.Lon_dec), axis=1)

In [ ]:
df_obs_latlon[df_obs_latlon.Country=='Germany']

## Recherche dans OpenStreetMap en fonction du nom de l'observatoire

In [ ]:
from geopy.geocoders import Nominatim

def get_location_info(address):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(address, addressdetails=True)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        raw_address = location.raw.get('address', {})
        country = raw_address.get('country', None)
        return latitude, longitude, country
    else:
        return None

In [ ]:
result = get_location_info('Xinglong Station')
result

In [ ]:
obs = []
# for address in df2.disc_facility.unique():
#     o ={}
#     print(address)
#     result = get_location_info(address)
#     if result:
#         print(f"Latitude: {result[0]}, Longitude: {result[1]}, Pays: {result[2]}")
#         o['Observatoire']=address
#         o['Latitude']=result[0]
#         o['Longitude']=result[1]
#         o['Country']=result[2]
#     else:
#         o['Observatoire']=address
#         o['Latitude']='NaN'
#         o['Longitude']='NaN'
#         o['Country']='NaN'
#         print("Location not found.")
#     print('--------------------------------')
#     obs.append(o)

In [ ]:
# pd.DataFrame(obs).to_csv('./Tables_traitees/Coordonnées_observatoires_from_OpenStreetMap.csv',index=False, sep='|')

In [ ]:
# df_obs_OSMap.merge(df_obs_latlon, how='left', left_on='Observatoire', right_on='FOUND').to_csv('./Tables_traitees/Observatoires_consolidés.csv',index=False, sep='|')

Rapprochement des noms, coordonnées des observatoires manuellement dans le fichier csv, "Observatoires_consolidés.csv"

## Création de la table contenant les distances entre les systèmes

In [19]:
pla_dist = pd.read_csv('.\\Tables_traitees\\planetes_avec_repere_cartesien_et_constellations.csv',sep=',')

In [20]:
pla_dist.head()

,loc_rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,disc_instrument,pl_orbper,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_eqt,st_spectype,st_teff,st_rad,st_mass,st_met,st_metratio,st_lum,st_logg,st_age,st_dens,ra,dec,sy_dist,x0,y0,z0,st_specclass,st_specsubdiv,st_lumclass,con
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,323.21000,12.2,1.09,4914.89849,15.464,14.90,0.238,NaN,G8 III,4874.0,13.76,2.09,-0.260,[Fe/H],1.978,2.45,NaN,NaN,185.178779,17.793252,93.1846,-88.364958,-8.008843,28.475644,G,8.0,III,Coma Berenices
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,516.21997,12.3,1.09,4684.81420,14.740,13.80,0.080,NaN,K4 III,4213.0,29.79,2.78,-0.020,[Fe/H],2.430,1.93,1.56,NaN,229.274595,71.823943,125.3210,-25.505212,-29.625963,119.067794,K,4.0,III,Ursa Minor
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,186.76000,13.1,1.16,1131.15130,3.559,2.76,0.000,NaN,K0 III,4888.0,11.55,1.78,-0.210,[Fe/H],1.840,2.55,4.50,NaN,352.824150,39.235837,75.4392,57.973692,-7.298956,47.716341,K,0.0,III,Andromeda
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,Radial Velocity,2002,W. M. Keck Observatory,10 m Keck I Telescope,HIRES Spectrometer,1765.03890,12.6,1.12,2559.47216,8.053,7.03,0.373,NaN,K0 V,5338.0,0.93,0.91,0.405,[Fe/H],-0.153,4.45,3.90,1.27393,242.602101,43.816362,17.9323,-5.954231,-11.487907,12.415415,K,0.0,V,Hercules
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,Radial Velocity,1996,Multiple Observatories,Multiple Telescopes,Multiple Instruments,798.50000,13.5,1.20,565.73740,1.780,1.26,0.680,NaN,G3 V,5750.0,1.13,1.08,0.060,[Fe/H],0.097,4.36,7.40,1.01103,295.465642,50.516824,21.1397,5.779530,-12.135760,16.315860,G,3.0,V,Cygnus


In [21]:
print(pla_dist.hostname.isna().sum())
print(pla_dist.hostname.shape)

0
(5539,)


In [22]:
pla_dist_uni = pla_dist.drop_duplicates(subset='hostname', keep="first").reset_index()

In [27]:
# Nombre de hostname différent
print(len(pla_dist.hostname.unique()))
pla_dist_uni.shape

4128


(4128, 44)

In [123]:
pla_dist_uni.x0 = round(pla_dist_uni.x0, 2)
pla_dist_uni.y0 = round(pla_dist_uni.y0, 2)
pla_dist_uni.z0 = round(pla_dist_uni.z0, 2)

### ATTENTION! le code chunk suivant prend beaucoup de temps à éxecuter 

In [137]:
dist_syst = list()
for i in range(pla_dist_uni.shape[0]):
    for j in range(pla_dist_uni.shape[0]):
        if j <= i or pd.isna(pla_dist_uni.x0[i]) or pd.isna(pla_dist_uni.x0[j]):
            continue 
        syst1 = pla_dist_uni.hostname[i]
        # print(pla_dist_uni.x0[i])
        # print(pla_dist_uni.x0[j])
        syst2 = pla_dist_uni.hostname[j]
        dist_x02 = np.power(pla_dist_uni.x0[i]-pla_dist_uni.x0[j], 2)
        dist_y02 = np.power(pla_dist_uni.y0[i]-pla_dist_uni.y0[j], 2)
        dist_z02 = np.power(pla_dist_uni.z0[i]-pla_dist_uni.z0[j], 2)
        dist = np.sqrt(int(dist_x02*100)/100. + int(dist_y02*100)/100. + int(dist_z02*100)/100.)
        dist_syst.append((syst1,syst2, dist))
        

In [138]:
len(dist_syst)

8431671

In [155]:
dist_syst_df = pd.DataFrame(dist_syst, columns=['syst1', 'syst2', 'dist'])

In [166]:
dist_syst_df.describe()

,dist
count,8.431671e+06
mean,1.106093e+03
std,1.543655e+03
min,0.000000e+00
25%,3.173068e+02
50%,6.401586e+02
75%,1.093667e+03
max,1.022510e+04


In [172]:
dist_syst_df[dist_syst_df.dist == 0]

,syst1,syst2,dist
2440680,HD 20781,HD 20782,0.0


In [185]:
pla_dist[pla_dist.hostname == 'HD 20782']

,loc_rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,disc_instrument,pl_orbper,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_eqt,st_spectype,st_teff,st_rad,st_mass,st_met,st_metratio,st_lum,st_logg,st_age,st_dens,ra,dec,sy_dist,x0,y0,z0,st_specclass,st_specsubdiv,st_lumclass,con
844,845,HD 20782 b,HD 20782,b,HD 20782,HIP 15527,TIC 144503325,Gaia DR2 5060104351007433472,2,5,Radial Velocity,2006,Anglo-Australian Telescope,3.9 m Anglo-Australian Telescope,UCLES Spectrograph,597.0643,13.6,1.21,472.83,1.4878,1.03,0.95,NaN,G1.5 V,5782.13,1.13,1.04,-0.054,[M/H],0.107,4.35,5.53,1.02483,50.016623,-28.854353,35.9864,20.2528,24.150573,-17.366488,G,1.5,V,Fornax


In [188]:
dist_syst_df.dist = round(dist_syst_df.dist, 2)

In [189]:
dist_syst_df.head()

,syst1,syst2,dist
0,11 Com,11 UMi,112.36
1,11 Com,14 And,147.59
2,11 Com,14 Her,84.03
3,11 Com,16 Cyg B,95.01
4,11 Com,17 Sco,119.78


In [260]:
dist_syst_df_light = dist_syst_df[dist_syst_df.dist <= 250.]

In [261]:
dist_syst_df_light.shape

(1615325, 3)

In [262]:
sys_uni1 = dist_syst_df_light.syst1.unique()
dist_syst_df_light.syst1.unique().shape

(3991,)

In [263]:
sys_uni2 = dist_syst_df_light.syst2.unique()
dist_syst_df_light.syst2.unique().shape

(3984,)

In [264]:
sum(pla_dist_uni.x0.isna())

21

In [265]:
pla_dist_uni.shape

(4128, 44)

In [266]:
count = 0
for i in range(pla_dist_uni.shape[0]):
    isok = False
    for j in range(sys_uni1.shape[0]):
        if sys_uni1[j] == pla_dist_uni.hostname[i]:
            count = count + 1
            isok = True
            break
    if isok == False:
        for j in range(sys_uni2.shape[0]):
            if sys_uni2[j] == pla_dist_uni.hostname[i]:
                count = count + 1
                isok = True
                break

print(count)
        

4062


In [268]:
pla_dist_uni.shape[0] - sum(pla_dist_uni.x0.isna()) - count

45

En supprimant toutes les distances supérieures à 250 parsecs, on perd 45 systèmes qui ne sont reliés à aucun autre à cause de leur éloignement.
Nous considéront ce chiffre suffisament faible par rapport au gain apporté en temps d'import sur Neo4j.

In [267]:
# dist_syst_df_light.to_csv('./Tables_traitees/Distances_systemes.csv',index=False, sep='|')

### Lecture du fichier resultat du calcul des distances

In [14]:
dist_syst_df_light = pd.read_csv('./Tables_traitees/Distances_systemes.csv',sep='|')

In [28]:
# Nombre théorique de distance à calculer
pla_dist_uni.shape[0]**2

17040384

In [30]:
# Nombre de distance calculées en ne retenant que les distance inférieure ou égales à 250 parsecs
dist_syst_df_light.shape[0]

1615325

In [31]:
# Nombre de distances éliminées
pla_dist_uni.shape[0]**2 - dist_syst_df_light.shape[0]

15425059

In [16]:
dist_syst_df_light.head()

,syst1,syst2,dist
0,11 Com,11 UMi,112.36
1,11 Com,14 And,147.59
2,11 Com,14 Her,84.03
3,11 Com,16 Cyg B,95.01
4,11 Com,17 Sco,119.78


In [18]:
print(len(dist_syst_df_light.syst1.unique()))
dist_syst_df_light.syst1.unique()

3991


array(['11 Com', '11 UMi', '14 And', ..., 'tau Gem', 'ups And', 'ups Leo'],
      dtype=object)

In [32]:
dist_syst_df_light.dist.describe()

count    1.615325e+06
mean     1.389277e+02
std      6.431250e+01
min      0.000000e+00
25%      8.545000e+01
50%      1.396900e+02
75%      1.937500e+02
max      2.500000e+02
Name: dist, dtype: float64